In [2]:
from src.datasets.chest import ChestXrayDataset
import torch
import numpy as np
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt
from src.datasets import get_dataset
from utils import make_exp_config
from src.preliminary_masks import get_latent_slice
from einops import reduce, rearrange, repeat
import os
print(os.path.abspath("."))


EXP_PATH = "../experiments/chestxray/compute_preliminary_chest_masks.py"
opt= make_exp_config(EXP_PATH)
#opt.dataset_args["load_segmentations"] = True
dataset = get_dataset(opt, split="train")
#dataset.add_preliminary_masks()


/vol/ideadata/ed52egek/pycharm/chest-distillation/notebooks


TypeError: get_dataset() got an unexpected keyword argument 'split'

In [3]:
import pandas as pd
data_entry_2017_w_bbox = pd.read_csv("/vol/ideadata/ed52egek/data/fobadiffusion/chestxray14/BBox_List_2017.csv").rename(columns={"Image Index":"idx"}).drop(columns=["Unnamed: 6", "Unnamed: 7", "Unnamed: 8"]).set_index("idx")
data_entry_2017_w_bbox = data_entry_2017_w_bbox.rename(columns={"Bbox [x": "bbox_x", "y": "bbox_y", "w": "bbox_w", "h]": "bbox_h"})
data_entry_2017_w_bbox

,Finding Label,bbox_x,bbox_y,bbox_w,bbox_h
idx,,,,,
00013118_008.png,Atelectasis,225.084746,547.019217,86.779661,79.186441
00014716_007.png,Atelectasis,686.101695,131.543498,185.491525,313.491525
00029817_009.png,Atelectasis,221.830508,317.053115,155.118644,216.949153
00014687_001.png,Atelectasis,726.237288,494.951420,141.016949,55.322034
00017877_001.png,Atelectasis,660.067797,569.780787,200.677966,78.101695
...,...,...,...,...,...
00029464_015.png,Atelectasis,198.940451,352.900747,615.537778,323.128889
00025769_001.png,Atelectasis,701.838229,572.491858,103.537778,63.715556
00016837_002.png,Atelectasis,140.913785,658.962969,271.928889,94.435556


In [4]:
data_entry_2020 = pd.read_csv("/vol/ideadata/ed52egek/data/fobadiffusion/chestxray14/Data_Entry_2017_v2020.csv").rename(columns={"Image Index":"idx"}).set_index("idx")


In [ ]:
for img_name in data_entry_2017_w_bbox.index:
    data_entry_2017_w_bbox
    data_entry_2020.loc[img_name]
    print(data_entry_2017_w_bbox.loc[img_name])



In [5]:
data_joined = data_entry_2020.join(data_entry_2017_w_bbox, how="left")


In [ ]:
for img_name in data_joined.index:
    data_entry_2020.loc[img_name]


print(len(data_joined) - len(data_entry_2017_w_bbox) - len(data_entry_2020))


In [ ]:
n = 980
data_joined[n:n+30]
#data_entry_2020[n:n+1]


In [ ]:
data_entry_2020[n:n+30]

In [6]:
data = data_joined[data_joined["View Position"] == "PA"]
#data = data[data["bbox_h"].notna()]
data.to_csv("data_paonly_joined.csv")


In [ ]:
massdata = data[data["Finding Label"]=="Mass"]
massdata

In [ ]:

entry = data_joined.loc[name]

print(entry)



## Filter for test data

In [ ]:
import torchxrayvision as xrv
import skimage, torch, torchvision
import torch
from einops import rearrange
import cv2
from torchvision.transforms import ToPILImage
from torchvision.utils import make_grid

massdata = data[data["Finding Label"]=="Infiltrate"]
print(len(massdata))
massdata

imgs = []
cnt = 0
for img_name in massdata.index:
    entry = massdata.loc[img_name]
    img = cv2.imread("/vol/ideadata/ed52egek/data/fobadiffusion/chestxray14/images/" + img_name)
    if img is None:
        #print(f"{img_name} not found")
        continue
    print(f"{img_name} found")

    x, y, w, h = int(entry["bbox_x"]), int(entry["bbox_y"]), int(entry["bbox_w"]), int(entry["bbox_h"])

    cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
    #cv2.rectangle(img, (y, x), (y+h, x+w), (255, 0, 0), 2)
    img = torch.tensor(img)
    imgs.append(rearrange(img, "h w c -> c h w"))

    cnt += 1
    if cnt == 25:
        break



img = torch.stack(imgs)
img = make_grid(img, nrow=5)
ToPILImage()(img).save("./bound.jpg")


In [ ]:
data_joined.loc["00000072_000.png"]

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(img)

In [21]:
path = "/vol/ideadata/ed52egek/data/fobadiffusion/chestxray14/images"

def file_to_list(path):
    lines = []
    with open(path) as fp:
        lines = fp.readlines()
    return lines

image_paths, splits = [], []
for entry in file_to_list(os.path.join(path + ".txt")):
    entry = entry.strip().split(" ")
    #image_paths.append(entry[0])
    #splits.append(entry[1])

    _, name = entry[0].split("/")


    try:
        data.loc[name]["split"] = entry[1]
    except KeyError:
        continue


In [18]:
data["split"] = ""

/tmp/ipykernel_946675/1267338619.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["split"] = ""


In [22]:
data.to_csv("data_paonly_joined.csv")

In [23]:
lo
data

,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Finding Label,bbox_x,bbox_y,bbox_w,bbox_h,split
idx,,,,,,,,,,,,,,,,
00000001_000.png,Cardiomegaly,0,1,57,M,PA,2682,2749,0.143,0.143,NaN,NaN,NaN,NaN,NaN,1
00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,NaN,NaN,NaN,NaN,NaN,1
00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,NaN,NaN,NaN,NaN,NaN,1
00000002_000.png,No Finding,0,2,80,M,PA,2500,2048,0.171,0.171,NaN,NaN,NaN,NaN,NaN,1
00000003_000.png,Hernia,7,3,81,F,PA,2582,2991,0.143,0.143,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
00030802_000.png,No Finding,0,30802,28,M,PA,2048,2500,0.168,0.168,NaN,NaN,NaN,NaN,NaN,0
00030803_000.png,No Finding,0,30803,42,F,PA,2048,2500,0.168,0.168,NaN,NaN,NaN,NaN,NaN,0
00030804_000.png,No Finding,0,30804,29,F,PA,2048,2500,0.168,0.168,NaN,NaN,NaN,NaN,NaN,0


In [ ]:
import json
f = open("/vol/ideadata/ed52egek/data/fobadiffusion/chestxray14/data.json")
jsondata = json.load(f)

In [38]:
jsondata


{'s50000014': {'caption': 'FINAL REPORT EXAMINATION: Chest radiograph INDICATION: year old woman presenting with weakness. Evaluate for pneumonia. TECHNIQUE: Semi upright AP radiograph view of the chest. COMPARISON: None. FINDINGS: Lung volumes are low. Retrocardiac opacity with silhouetting of the left hemidiaphragm and lateral border of the descending aorta is nonspecific and could reflect any of a combination of atelectasis, focal pneumonia or even a small effusion. Right infrahilar opacity with slight indistinctness of the right heart border could reflect infection in the appropriate clinical scenario. Apparent elevation of the right hemidiaphragm may be related to positioning and technique versus volume loss. There is cardiomegaly, but no CHF. Aortic calcifications are present. No pneumothorax. IMPRESSION: 1. Nonspecific retrocardiac and right middle lobe opacities could reflect atelectasis and or infection in the appropriate clinical situation. 2. There is cardiomegaly without CH